In [1]:
import os
import numpy as np
import pandas as pd
import glob

Step 1. 형태소 분석기 변경해 보기 - Mecab 이외에도 Hannanum, Kkma, Komoran, Okt


Step 2. 불용어 추가해 보기 - 직접 뉴스를 열어보면서 불용어가 총 50개가 될 때까지 추가, 추가 전후 모델성능 비교


Step 3. 다른 날짜 데이터 추가해 보기 - 미 수집한 2020년 5월 6일 데이터와 추가로 수집된 데이터를 적절히 합치려면? 적절히 데이터를 섞기위해 수집고민하기

# 데이터 가져오기

In [2]:
glob.glob(os.getenv('HOME')+'/aiffel/news_crawler/*.csv')

['/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20200215.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20200530.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20200914.csv',
 '/home/jin/aiffel/news_crawler/news_data.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20200429.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20200129.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20200819.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20200614.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20201024.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20200715.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20200315.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20201119.csv']

In [3]:
# map(func, iterable) 후 pd.concat로 DataFrame결합
df = pd.concat(map(pd.read_csv, glob.glob(os.getenv('HOME')+'/aiffel/news_crawler/*.csv')))

In [4]:
os.getcwd()

'/home/jin/workplace'

In [5]:
df

,news,code
0,[파이낸셜뉴스] 고가주택에 대한 대출규제를 담은 12.16대책 이후 서울 아파트 시...,경제
1,플룸테크 JTI코리아 제공 플룸테크 JTI코리아 제공\n\n[파이낸셜뉴스] 전자담배...,경제
2,[파이낸셜뉴스] 이아이디가 2차전지 자회사의 실적호전에 힘입어 지난해 큰 폭의 턴어...,경제
3,본설 '배달 패키지' 본설 '배달 패키지'\n\n[파이낸셜뉴스] 외식 배달 전성시대...,경제
4,신종 코로나바이러스 감염증 공포가 확산하는 가운데 29일 인천국제공항 제1터미널 입...,경제
...,...,...
1595,중소벤처기업부는 19일 서울 여의도 중소기업중앙회 구내 나들가게에서 '스마트슈퍼 2...,IT/과학
1596,[서울=뉴시스] 이재은 기자 = 국내 대기업과 중견기업 상당수가 정부의 디지털 뉴딜...,IT/과학
1597,"(지디넷코리아=이도원 기자)넷마블(대표 권영식, 이승원)은 모바일 다중접속역할수행게...",IT/과학
1598,SK텔레콤에서 AI 개발을 담당하고 있는 현직 전문가들이 19일 온라인으로 진행된 ...,IT/과학


# 데이터 전처리

In [6]:
df1 = df.copy()

* news column의 **문자열**을 한글이 아닌것(^)모두 빈칸으로

In [7]:

df1['news'] = df['news'].str.replace('[^ㄱ-ㅎ가-힣ㅏ-ㅣ]',' ')

In [8]:
df1

,news,code
0,파이낸셜뉴스 고가주택에 대한 대출규제를 담은 대책 이후 서울 아파트 시...,경제
1,플룸테크 코리아 제공 플룸테크 코리아 제공 파이낸셜뉴스 전자담배 시...,경제
2,파이낸셜뉴스 이아이디가 차전지 자회사의 실적호전에 힘입어 지난해 큰 폭의 턴어...,경제
3,본설 배달 패키지 본설 배달 패키지 파이낸셜뉴스 외식 배달 전성시대를 ...,경제
4,신종 코로나바이러스 감염증 공포가 확산하는 가운데 일 인천국제공항 제 터미널 입...,경제
...,...,...
1595,중소벤처기업부는 일 서울 여의도 중소기업중앙회 구내 나들가게에서 스마트슈퍼 ...,IT/과학
1596,서울 뉴시스 이재은 기자 국내 대기업과 중견기업 상당수가 정부의 디지털 뉴딜...,IT/과학
1597,지디넷코리아 이도원 기자 넷마블 대표 권영식 이승원 은 모바일 다중접속역할수행게...,IT/과학
1598,텔레콤에서 개발을 담당하고 있는 현직 전문가들이 일 온라인으로 진행된 ...,IT/과학


* 데이터에 null존재 확인

In [9]:
df1.isnull().sum()

news    0
code    0
dtype: int64

* 중복된 샘플 제거

In [10]:
df1.drop_duplicates(subset=['news'], inplace=True, ignore_index=True)

* 중복 제거결과 18820개의 뉴스가 15455개로 정리됨

In [11]:
print(df1['news'][0])

 파이낸셜뉴스  고가주택에 대한 대출규제를 담은      대책 이후 서울 아파트 시장이 양극화되는 분위기다  고가 아파트가 밀집한 강남권은 매수심리가 위축되면서 재건축 중심의 하락세를 보였다  반면 비강남권의 경우 중저가 아파트가 많은 노원  관악  도봉구 등에 수요가 유입되면서 집값 오름세를 이어갔다  수도권도 교통호재로 강남권 접근성이 개선되는 수원  용인 등 경기 남부권 일부 지역을 중심으로 집값이 큰 폭으로 올랐다   일 부동산   에 따르면 이번주 서울 아파트 매매가격은      를 기록했다  재건축은 강남구 대치동 은마와 송파구 잠실동 주공 단지 등 주요 단지가 하향 조정되면서       떨어졌다  일반 아파트는       올라 상승폭이 둔화됐다  신도시는       올랐다  경기 인천은 중저가 아파트로 수요가 유입되면서       상승했다 전세시장은 신종코로나 코로나   의 영향으로 국지적으로 문의가 줄었지만  서울은 매물품귀가 지속되며       상승했다  신도시는       올라 전주와 비슷한 흐름을 이어갔다  경기 인천은       상승해 지난주       보다 오름폭이 커졌다 서울은 비강남권의 상승세가 이어지면서 지역별로  노원         관악         성북         도봉         광진         강서         구로        순으로 올랐다  노원은 광운대 역세권 개발  동북선 경전철 호재가 있는 월계동 미륭 미성 삼호 차를 비롯해 상계동 주공 단지  불암현대 등이    만원     만원 상승했다 반면 고가 아파트 밀집지역인 송파          동작          강남        은 떨어졌다  송파는 잠실동 주공 단지와 레이크팰리스  신천동 잠실파크리오가     만원     만원 하락했다  동작은     년 입주한 흑석동 아크로리버하임이     만원 하향 조정됐다  강남은 대치동 은마  한보미도맨션   차 등 재건축 단지가    만원     만원 내렸다 신도시는  동탄         광교         분

# 데이터 탐색

In [13]:
df1.code.value_counts(ascending=True)

IT/과학    3475
사회       3932
경제       3953
생활/문화    4104
Name: code, dtype: int64

In [14]:
# 데이터 실제 갯수 확인
# size() - 각 그룹의 사이즈를 취득할 수 있습니다.
print(df1.groupby('code').size().reset_index(name='count'))

    code  count
0  IT/과학   3475
1     경제   3953
2     사회   3932
3  생활/문화   4104


* 토큰화

In [15]:
from konlpy.tag import Okt, Hannanum, Kkma, Komoran

In [16]:
tokenizer1 = Komoran()

In [17]:
tokenizer1.morphs

<bound method Komoran.morphs of <konlpy.tag._komoran.Komoran object at 0x7fdb996b2690>>

* 불용어 제거

In [23]:
# 시간관걔상 기존stopword사용
stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스']

In [24]:
# def preprocessing(data):
#     text_data=[]
    
#     for sentence in data:
#         temp_data=[]
#         # 토큰화
#         temp_data = tokenizer1.morphs(sentence)
        
#         # 불용어 제거
#         temp_data = [word for word in temp_data if not word in stopwords]
#         text_data.append(temp_data)
        
#     text_data = list(map(' '.join, text_data))  #.join -- list to str
    
#     return text_data
    

In [26]:
def preprocess(data):
    text_data=[]
    for line in data:
        
        word_data = tokenizer1.morphs(line)
           
        temp_data = [word for word in word_data if not word in stopwords]
        text_data.append(word_data)
        
    text_data = list(map(' '.join, text_data))  #.join -- list to str
        
    return text_data
   
    
    

In [33]:
test_set = preprocess(df1.news)

In [29]:
df1.head()

,news,code
0,파이낸셜뉴스 고가주택에 대한 대출규제를 담은 대책 이후 서울 아파트 시...,경제
1,플룸테크 코리아 제공 플룸테크 코리아 제공 파이낸셜뉴스 전자담배 시...,경제
2,파이낸셜뉴스 이아이디가 차전지 자회사의 실적호전에 힘입어 지난해 큰 폭의 턴어...,경제
3,본설 배달 패키지 본설 배달 패키지 파이낸셜뉴스 외식 배달 전성시대를 ...,경제
4,신종 코로나바이러스 감염증 공포가 확산하는 가운데 일 인천국제공항 제 터미널 입...,경제


In [39]:
df1.shape

(15464, 2)

# 머신러닝 사용하기

In [44]:
# train_test_split, metrics , naivebaise
# feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [38]:
len(test_set)

15464

In [40]:
#- 훈련 데이터와 테스트 데이터를 분리합니다.
X_train, X_test, y_train, y_test = train_test_split(test_set, df1['code'], random_state = 0)

In [41]:
print('훈련용 뉴스 기사의 개수 :', len(X_train))
print('테스트용 뉴스 기사의 개수 : ', len(X_test))
print('훈련용 레이블의 개수 : ', len(y_train))
print('테스트용 레이블의 개수 : ', len(y_test))

훈련용 뉴스 기사의 개수 : 11598
테스트용 뉴스 기사의 개수 :  3866
훈련용 레이블의 개수 :  11598
테스트용 레이블의 개수 :  3866


## vectorize, TFIDF

In [42]:
#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [45]:
#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [46]:
# 텍스트를 입력하면 자동으로 TF-IDF벡터로 바꾸는 전처리 함수
# count_vect의 정보가 이미 위에서 저장되있으므로 fit 필요없음??

def tfidf_vectorizer(data):
    data_counts = count_vect.transform(data)
    data_tfidf = tfidf_transformer.transform(data_counts)
    return data_tfidf

* test set에 예측적용

In [47]:
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.78      0.77      0.77       868
          경제       0.70      0.72      0.71       995
          사회       0.74      0.82      0.78       970
       생활/문화       0.80      0.70      0.75      1033

    accuracy                           0.75      3866
   macro avg       0.75      0.75      0.75      3866
weighted avg       0.75      0.75      0.75      3866



# 머신러닝 성능향상?

## 형태소 분석기 변경

In [48]:
tokenizer2 = Kkma()

In [49]:
def preprocess_k(data):
    text_data=[]
    for line in data:
        
        word_data = tokenizer2.morphs(line)
           
        temp_data = [word for word in word_data if not word in stopwords]
        text_data.append(word_data)
        
    text_data = list(map(' '.join, text_data))  #.join -- list to str
        
    return text_data

In [50]:
test_set2 = preprocess_k(df1.news)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(test_set2, df1['code'], random_state = 0)

In [52]:
#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [53]:
#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [55]:
# 텍스트를 입력하면 자동으로 TF-IDF벡터로 바꾸는 전처리 함수
# count_vect의 정보가 이미 위에서 저장되있으므로 fit 필요없음??

def tfidf_vectorizer2(data):
    data_counts = count_vect.transform(data)
    data_tfidf = tfidf_transformer.transform(data_counts)
    return data_tfidf

In [56]:
y_pred = clf.predict(tfidf_vectorizer2(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.77      0.77      0.77       868
          경제       0.69      0.71      0.70       995
          사회       0.75      0.81      0.78       970
       생활/문화       0.79      0.70      0.74      1033

    accuracy                           0.75      3866
   macro avg       0.75      0.75      0.75      3866
weighted avg       0.75      0.75      0.75      3866



* Kkma로 토큰화 결과 정확도엔 큰차이가 없음

In [ ]:
# 데이터추가후 실행하려했으나 시간이 많이 소요되어 미완